In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
file_path = "/content/drive/MyDrive/Dataset Idealista/pisosBarcelona-04-04-2025.csv"
df = pd.read_csv(file_path, encoding='latin1')

ELIMINAR INNECESARIAS

In [ ]:
import pandas as pd

# Asumiendo que tu DataFrame se llama 'df'
columnas_a_eliminar = [
    'thumbnail', 'externalReference', 'showAddress', 'distance',
    'change', 'suggestedTexts', 'savedAd', 'notes'
]

# Elimina las columnas y actualiza df
df = df.drop(columns=columnas_a_eliminar, errors='ignore')

# Opcional: verifica las columnas restantes
# print(df.columns)

CONVERTIR JSON

In [ ]:
import pandas as pd
import ast # Para evaluar strings como literales de Python de forma segura

# Asumiendo que tu DataFrame se llama 'df'

# --- Helper Function ---
def safe_get_from_str_dict(string_dict, key, default=None):
    """
    Intenta parsear un string que representa un dict y obtener un valor.
    Maneja errores y valores no string/NaN.
    """
    try:
        # Si no es string o es NaN, retorna el default directamente
        if not isinstance(string_dict, str) or pd.isna(string_dict):
            return default
        # Parsea el string a un diccionario
        data = ast.literal_eval(string_dict)
        # Si es un diccionario, obtén el valor; si no, retorna default
        return data.get(key, default) if isinstance(data, dict) else default
    except (ValueError, SyntaxError):
        # Si el string no es un literal válido (ej: mal formado)
        return default

# --- Extracción ---

# 1. De 'detailedType' obtener 'subTypology'
df['subTypology'] = df['detailedType'].apply(lambda x: safe_get_from_str_dict(x, 'subTypology'))

# 2. De 'parkingSpace' obtener 'hasParking' y 'isParkingIncludedInPrice'
# Usamos default=False y convertimos a booleano explícitamente
df['hasParking'] = df['parkingSpace'].apply(
    lambda x: safe_get_from_str_dict(x, 'hasParkingSpace', default=False)
).astype(bool)
df['isParkingIncludedInPrice'] = df['parkingSpace'].apply(
    lambda x: safe_get_from_str_dict(x, 'isParkingSpaceIncludedInPrice', default=False)
).astype(bool)

# 3. De 'highlight' obtener 'highlight_type'
df['highlight_type'] = df['highlight'].apply(lambda x: safe_get_from_str_dict(x, 'groupDescription'))

# --- Limpieza ---

# 4. Eliminar las columnas originales
df = df.drop(columns=['detailedType', 'parkingSpace', 'highlight'], errors='ignore')

# --- Opcional: Verificar las nuevas columnas ---
print(df[['subTypology', 'hasParking', 'isParkingIncludedInPrice', 'highlight_type']].head())
print(df[['hasParking', 'isParkingIncludedInPrice']].dtypes) # Verificar que sean booleanas

In [ ]:

# --- Opcional: Verificar las nuevas columnas ---
print(df[['subTypology', 'hasParking', 'isParkingIncludedInPrice', 'highlight_type']].head())
print(df[['hasParking', 'isParkingIncludedInPrice']].dtypes) # Verificar que sean booleanas

        subTypology  hasParking  isParkingIncludedInPrice highlight_type
0              None       False                     False           None
1              None       False                     False      Destacado
2              None       False                     False      Destacado
3  independantHouse       False                     False           None
4              None       False                     False           None
hasParking                  bool
isParkingIncludedInPrice    bool
dtype: object


In [ ]:
df.to_csv('/content/drive/MyDrive/Dataset Idealista/pisosBarcelona-04-04-2025-clean.csv', index=False)